# ABM Results

In [71]:
import seaborn as sns
sns.set('talk')
sns.set_style('ticks')

import numpy as np
import networkx as nx
import datetime
import tqdm
from typing import NamedTuple
import h5py

import time
import random
import pandas as pd
import pickle
from collections import Counter, defaultdict
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bernoulli, binom
sns.set('talk')
sns.set_style('ticks')

import sys

sys.path.append( '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/src/d04_modelling' )

from abm import *
from ipynb.fs.defs.ABM import group_peaks_and_daterange

#load in search hashtags
with open('../references/search_hashtags.txt', 'r') as f:
    search_hashtags = f.readlines()
    search_hashtags = [i.replace('\n', '') for i in search_hashtags]
    search_hashtags = [i.replace('#', '') for i in search_hashtags]
    search_hashtags = [i.lower() for i in search_hashtags]
    search_hashtags.remove('وأناكمان')

read in results

In [22]:
group_num = 2
results_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/06_reporting/ABM_output_group_2.obj'

activity_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/03_processed/activity_counts.hdf5'

peak_analysis_file = '/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_analysis.hdf5'

most_prominent_peaks, group_date_range, daterange_length = group_peaks_and_daterange(peak_analysis_file, group_num)

with open(results_file, 'rb') as f:
    raw_results = pickle.load(f)

each entry is a tuple of the configurations given and the abm result.

In [7]:
raw_results[0]

({'experimentation_chance': 0.1,
  'initial_activity_threshold': 1,
  'interact_prob': 0.1,
  'interact_prob_multiplier': 1.1,
  'interact_threshold': 1,
  'model_num': 4,
  'peak_delta_init': 7,
  'search_hashtag_propensity': 0.1},
 {'1000012910': <abm.Agent at 0x7f73aa0ce850>,
  '1000030383969316865': <abm.Agent at 0x7f73aa0ce430>,
  '1000044199436259328': <abm.Agent at 0x7f73aa0ce4f0>,
  '1000093693385347072': <abm.Agent at 0x7f73aa0ce160>,
  '1000433909472092160': <abm.Agent at 0x7f73aa0ce3d0>,
  '1000514947942637568': <abm.Agent at 0x7f73aa0ce700>,
  '1000674423500951552': <abm.Agent at 0x7f73aa0ce040>,
  '1000870773945446402': <abm.Agent at 0x7f73aa0ce250>,
  '1000996505396723712': <abm.Agent at 0x7f73aa0cefa0>,
  '1001064352735166466': <abm.Agent at 0x7f73aa0ce190>,
  '1001073393901465600': <abm.Agent at 0x7f73aa0edc40>,
  '1001207735613116417': <abm.Agent at 0x7f73aa0edcd0>,
  '1001349187': <abm.Agent at 0x7f73aa0ed6d0>,
  '1001398273565970432': <abm.Agent at 0x7f73aa0eddc0>,
 

In [25]:
def print_abm_results(agents_dict, model_num=None):

    output_dict={}

    total = len(agents_dict)
    # num_supporting = 0
    # num_not_supporting = 0

    for user_id, agent in agents_dict.items():
        output_dict[user_id] = agent.supporting_metoo_dict

    output_df = pd.DataFrame.from_dict(output_dict, orient='index').reset_index()

    num_supporting = output_df.iloc[:,1:].sum(axis=0)
    num_supporting = num_supporting[num_supporting>0]
    num_not_supporting = (output_df==False).sum(axis=0)

    # print(num_supporting)
    # print(num_not_supporting)
    return output_df


def reference_results(activity_file, agents, group_num):
    act_val = {}
    for user_id, agent in agents.items():
        # obtain user activity
        act_val[user_id] = {}
        with h5py.File(activity_file, 'r') as f:
            activity = f[f'group_{group_num}'][user_id]['hashtagged'][:]
            feature_order = f[f'group_{group_num}'][user_id]['hashtagged'].attrs['feature_order']
            feature_order = feature_order.split(';')
            # act_val[user_id] = np.sum(activity[:,-int(daterange_length/2):])

            for hashtag_in_period in most_prominent_peaks:
                hashtag_in_period_index = feature_order.index(hashtag_in_period)

                # obtain the index offset from the detected peak of the hashtag to collect initial time window.
                peak_index_index = (datetime.datetime.strptime(group_date_range.end, '%Y-%m-%d')-most_prominent_peaks[hashtag_in_period]).days
                # offset_index -= peak_delta_init
                # offset_index = max(0,offset_index)+1
                # print(f'Offset for {hashtag_in_period} is {offset_index}')

                act_val[user_id][hashtag_in_period_index]= np.sum(activity[hashtag_in_period_index,-peak_index_index-1:])

    act_val = pd.DataFrame.from_dict(act_val, orient='index').reset_index()
    act_val.columns = ['user_id'] + list(most_prominent_peaks.keys())

    return act_val


In [66]:
act_val = reference_results(activity_file, raw_results[param_grid_num][1], group_num)
act = (act_val.iloc[:,1:]>2).sum(axis=0).to_frame().reset_index()
act.columns = ['index', 'actual']

In [67]:
param_grid_num = 5

In [68]:
raw_results[param_grid_num][0]

{'experimentation_chance': 0.1,
 'initial_activity_threshold': 1,
 'interact_prob': 0.1,
 'interact_prob_multiplier': 1.1,
 'interact_threshold': 2,
 'model_num': 6,
 'peak_delta_init': 7,
 'search_hashtag_propensity': 0.1}

In [69]:
res = print_abm_results(raw_results[param_grid_num][1],raw_results[param_grid_num][0]['model_num'])
num_supporting = res.iloc[:,1:].sum(axis=0)
num_supporting = num_supporting[num_supporting>0]
num_supporting = num_supporting.to_frame().reset_index()
num_supporting.columns = ['index', 'abm']

In [70]:
comparison = act.merge(num_supporting, on='index', how='right').fillna(0)
comparison

,index,actual,abm
0,metoo,752.0,762
1,moiaussi,11.0,4
2,sendeanlat,0.0,1
3,wetoo,7.0,18
4,niunamenos,266.0,5
5,noustoutes,530.0,29


# Plotting simulations

In [77]:
for user in list(raw_results[param_grid_num][1].keys()):
    user_to_examine = raw_results[param_grid_num][1][user]
    if user_to_examine.support_tracker.sum() > 0:
        break

In [81]:
user_to_examine.probability_matrix


array([[1. , 0.9, 0.9, ..., 0.9, 0.9, 0.9],
       [1. , 1. , 1. , ..., 1. , 1. , 1. ],
       [1. , 1. , 1. , ..., 1. , 1. , 1. ],
       ...,
       [1. , 1. , 1. , ..., 1. , 1. , 1. ],
       [1. , 1. , 1. , ..., 1. , 1. , 1. ],
       [1. , 1. , 1. , ..., 1. , 1. , 1. ]])

In [ ]:
user_to_examine = raw_results[param_grid_num][1][list(raw_results[param_grid_num][1].keys())[0]]
# usertype = 'low'

class daterange(NamedTuple):
    start: str
    end: str

# obtain user activity
with h5py.File(activity_file, 'r') as f:
    activity = f[f'group_{group_num}'][user_to_examine]['hashtagged'][:]
    feature_order = f[f'group_{group_num}'][user_to_examine]['hashtagged'].attrs['feature_order']
    feature_order = feature_order.split(';')

with h5py.File('/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/FAS_peak_analysis.hdf5', 'r') as f:
    FAS_peaks = f['peak_detections']
    x = f['segments']['selected_ranges'][int(group_num)-1]
    group_date_range = daterange(
        start = x[0].decode(),
        end = x[1].decode()
    )


daterange = pd.date_range(start=group_date_range.start, end=group_date_range.end)
if group_num == 1:
    daterange = pd.date_range(start=group_date_range.start, end='2018-05-30')

plot_user_df = pd.DataFrame({
    'created_at' : daterange})

for index, feature in enumerate(feature_order):
    plot_user_df[f'activity_{feature}'] = activity[index,:]

plot_user_df=pd.wide_to_long(plot_user_df, stubnames='activity', i='created_at', j='ht', sep = '_', suffix='.*').reset_index()

user_df = df[df['author_id']==user_to_examine]
user_df = user_df.groupby(['created_at', 'ht']).count()['tweet_id'].reset_index()
user_df['created_at'] = pd.to_datetime(user_df['created_at'])
final_user_plot_df = user_df.merge(plot_user_df, on = ['created_at','ht'], how='left')
final_user_plot_df.columns = ['created_at', 'Hashtag', 'Interaction Count', 'Hashtag Protest Network Activity']

# final_user_plot_df['interaction_count'] = final_user_plot_df['activity']-final_user_plot_df['tweet_id']
# final_user_plot_df = final_user_plot_df.drop(['tweet_id'])

df[df['author_id']==user_to_examine]

temp = df.groupby(['author_id','ht']).count().reset_index()
temp = temp[['author_id', 'ht']].groupby('author_id').count()
temp[temp['ht']>1]

pd.melt(final_user_plot_df, ['created_at','Hashtag'])

fig = plt.figure(figsize = (16,9))

# Draw line plot of size and total_bill with parameters and hue "day"
sns.lineplot(
    x = "created_at", y = "value", hue='variable',data=pd.melt(final_user_plot_df, ['created_at','Hashtag']))
plt.title("Interactions per day per hashtag", fontsize = 20)
plt.xlabel("Date", fontsize = 15)
plt.ylabel("Interactions", fontsize = 15)
plt.savefig(f'{plot_save_path}user_{usertype}_output_{group_num}.png')
plt.show();